In [1]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import pandas as pd
from nltk.stem.snowball import SnowballStemmer
import string
from string import digits
nltk.download('punkt')
nltk.download('stopwords')
import xgboost as xgb
import numpy as np
import sklearn as sk
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import ensemble, metrics, model_selection, naive_bayes

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Radames\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Radames\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


c:\users\radames\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df_train = pd.read_csv("../data/train.csv")
df_test = pd.read_csv("../data/test.csv")
# 3 столбца - id, text, author
df_train.head(n=3)

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP


In [3]:
remove_digits = str.maketrans('', '', digits)
def tokenize_stem(file_text):
    #firstly let's apply nltk tokenization
    file_text = file_text.translate(remove_digits)
    
    tokens = nltk.word_tokenize(file_text)

    #let's delete punctuation symbols
    tokens = [i for i in tokens if ( i not in string.punctuation )]

    #deleting stop_words
    stop_words = stopwords.words('english')
    tokens = [i for i in tokens if ( i not in stop_words )]

    #cleaning words
    stemmer = SnowballStemmer("english")
    
    tokens = [stemmer.stem(i) for i in tokens]

    return tokens

In [4]:
df_train['cleaned_text'] = df_train.text.apply(tokenize_stem)
df_train['cleaned_text_string'] = df_train.cleaned_text.apply(' '.join)
df_train.head(n=3)
eng_stopwords = set(stopwords.words("english"))

In [5]:
df_train['length']=df_train['cleaned_text_string'].apply(len)
df_train["num_words"] = df_train["text"].apply(lambda x: len(str(x).split()))
df_train["num_unique_words"] = df_train["text"].apply(lambda x: len(set(str(x).split())))
df_train["num_punctuations"] =df_train['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
df_train["num_words_upper"] = df_train["text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
df_train["num_words_title"] = df_train["text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
df_train["mean_word_len"] = df_train["text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
df_train["num_stopwords"] = df_train["text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
df_train.head(n=3)

,id,text,author,cleaned_text,cleaned_text_string,length,num_words,num_unique_words,num_punctuations,num_words_upper,num_words_title,mean_word_len,num_stopwords
0,id26305,"This process, however, afforded me no means of...",EAP,"[this, process, howev, afford, mean, ascertain...",this process howev afford mean ascertain dimen...,145,41,35,7,2,3,4.658537,19
1,id17569,It never once occurred to me that the fumbling...,HPL,"[it, never, occur, fumbl, might, mere, mistak]",it never occur fumbl might mere mistak,38,14,14,1,0,1,4.142857,8
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,"[in, left, hand, gold, snuff, box, caper, hill...",in left hand gold snuff box caper hill cut man...,116,36,32,5,0,1,4.583333,16


In [6]:
df_hpl=df_train[df_train['author']=='HPL']
df_hpl.describe()

,length,num_words,num_unique_words,num_punctuations,num_words_upper,num_words_title,mean_word_len,num_stopwords
count,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000,5635.000000
mean,93.395386,27.799645,24.437977,3.206921,0.500266,2.334694,4.625193,12.940728
std,51.075096,14.123252,11.053739,2.108637,0.852313,2.041579,0.554917,6.855502
min,7.000000,4.000000,3.000000,1.000000,0.000000,0.000000,2.222222,0.000000
25%,58.000000,18.000000,17.000000,2.000000,0.000000,1.000000,4.258065,8.000000
50%,85.000000,26.000000,23.000000,3.000000,0.000000,2.000000,4.600000,12.000000
75%,118.000000,35.000000,30.000000,4.000000,1.000000,3.000000,4.961538,16.500000
max,561.000000,147.000000,102.000000,28.000000,10.000000,39.000000,7.833333,62.000000


In [7]:
df_eap=df_train[df_train['author']=='EAP']
df_eap.describe()

,length,num_words,num_unique_words,num_punctuations,num_words_upper,num_words_title,mean_word_len,num_stopwords
count,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000,7900.000000
mean,81.543165,25.442405,21.894937,4.096329,0.553291,2.102405,4.644952,12.617722
std,60.100183,18.567706,13.727397,3.573788,0.892966,2.052241,0.631340,9.540922
min,5.000000,2.000000,2.000000,1.000000,0.000000,0.000000,2.000000,0.000000
25%,40.000000,12.000000,12.000000,2.000000,0.000000,1.000000,4.250000,6.000000
50%,66.000000,21.000000,19.000000,3.000000,0.000000,1.000000,4.600000,10.000000
75%,106.000000,33.000000,29.000000,5.000000,1.000000,2.000000,5.000000,17.000000
max,925.000000,267.000000,155.000000,71.000000,15.000000,43.000000,11.000000,135.000000


In [8]:
df_mws=df_train[df_train['author']=='MWS']
df_mws.describe()

,length,num_words,num_unique_words,num_punctuations,num_words_upper,num_words_title,mean_word_len,num_stopwords
count,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000,6044.000000
mean,86.124586,27.417273,23.544672,3.833719,0.751489,2.124255,4.598182,13.742224
std,71.976281,23.134440,14.925835,2.840625,1.203636,1.759572,0.561558,12.080172
min,4.000000,2.000000,2.000000,1.000000,0.000000,0.000000,2.666667,0.000000
25%,48.000000,15.000000,14.000000,2.000000,0.000000,1.000000,4.250000,7.000000
50%,74.000000,23.000000,21.000000,3.000000,0.000000,2.000000,4.560791,12.000000
75%,108.000000,34.000000,30.000000,5.000000,1.000000,3.000000,4.907156,17.000000
max,2715.000000,861.000000,429.000000,59.000000,27.000000,46.000000,10.500000,435.000000


In [9]:
wordset=set()

In [10]:
#делаю сет со всеми словами
for i in df_train.index:
    wordset |= set(df_train['cleaned_text'][i])
wordlist=list(wordset)

In [11]:
#делаю фрейм со словами
df_word=pd.DataFrame(columns=["word", "mws", "eap", "hpl", "all"])
df_word["word"]=wordlist
df_word["mws"]=0
df_word["eap"]=0
df_word["hpl"]=0
df_word["all"]=0
df_word.head()

,word,mws,eap,hpl,all
0,tame,0,0,0,0
1,bereav,0,0,0,0
2,unlaw,0,0,0,0
3,hose,0,0,0,0
4,baudelair,0,0,0,0


In [12]:
# как мы будем эту штуку правильнее делать (возможно это жуткий костыль), я хз
# сначала создаем словарь где ключ - уникальное слово, а значение - его порядковый номер
# затем создаем разреженную матрицу, которую заполняем в зависимости от порядковых номеров 
word_dict = {}



In [13]:
#делаю сет со всеми словами
# и сразу заготовку под шапку(потом увидишь зачем)
counter = 0
head = []

for wordlist in df_train['cleaned_text']:
    for word in wordlist:
        if word not in word_dict:
            head.append(word)
            word_dict[word] = counter
            counter += 1


In [14]:
# видоизменять колонки в pandas руками по одному значению в строке или столбце - очень плохая идея
# колонка это numpy.ndarray, а значит при каждой итерации она будет пересоздаваться
# что угробит производительность
# делаем значит так. считаем где сколько и где встречались отдельные слова, затем создаем строку за строкой для 
# каждого предложения

list_of_lists = []

for wordlist in df_train['cleaned_text']:
    row = [0 for i in range(len(word_dict))]
    for word in wordlist:
        row[word_dict[word]] += 1
    list_of_lists.append(row)



In [15]:
print(len(list_of_lists))

19579


In [16]:
# ... и для того чтобы посмотреть встречаемость того или иного слова по авторам добавим такую колонку

count_frame = pd.DataFrame(list_of_lists)
count_frame['author'] = df_train['author']



In [17]:
# ... и теперь нормальную шапку делаем

count_frame.columns = head + ['author']


In [18]:
print(count_frame.head())

   this  process  howev  afford  mean  ascertain  dimens  dungeon  i  might  \
0     1        1      1       1     1          1       1        1  2      1   
1     0        0      0       0     0          0       0        0  0      1   
2     0        0      0       0     0          0       0        0  0      0   
3     0        0      0       0     0          0       0        0  0      0   
4     0        0      0       0     0          0       0        0  0      0   

    ...    aegidus  burr  bentley  waltzer  binder  brusqueri  adriat  ancona  \
0   ...          0     0        0        0       0          0       0       0   
1   ...          0     0        0        0       0          0       0       0   
2   ...          0     0        0        0       0          0       0       0   
3   ...          0     0        0        0       0          0       0       0   
4   ...          0     0        0        0       0          0       0       0   

   agir  author  
0     0     EAP  
1 

In [19]:
# Пока объединим все, потом может быть будем использовать
col=list(count_frame.columns)
col[-1]='author_name'
count_frame.columns=col
pivot_col=pd.pivot_table(count_frame, aggfunc=np.sum, values=col, index=['author_name'])

In [20]:
#Убираем лишние слова, которые не учли раньше
col=list(pivot_col.columns)
col2=[string for string in col if (string[0]!='"' and string[0]!="'"
                                  and string[0]!='.' and string[0]!='`'
                                   and len(string)>3 and '.' not in string)]
col=[]
pivot_col=pivot_col[col2]
pivot_col.head()

,aaem,aback,abaft,abandon,abaout,abas,abash,abat,abbey,abbrevi,...,æmilianus,æneid,ærial,æronaut,ærostat,æschylus,élite,émeut,οἶδα,υπνος
author_name,,,,,,,,,,,,,,,,,,,,,
EAP,1,2,0,22,0,2,1,2,3,2,...,0,0,1,3,1,1,1,1,0,0
HPL,0,0,0,17,24,0,1,3,0,0,...,2,1,0,0,0,0,0,0,2,1
MWS,0,0,1,9,0,0,0,1,2,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
#Create pivot
pivot_col=pivot_col.append(pivot_col.sum(), ignore_index=True)
pivot_col.index=['EAP', 'HPL', 'MWS', 'SUMA']
pivot_col.head()

,aaem,aback,abaft,abandon,abaout,abas,abash,abat,abbey,abbrevi,...,æmilianus,æneid,ærial,æronaut,ærostat,æschylus,élite,émeut,οἶδα,υπνος
EAP,1,2,0,22,0,2,1,2,3,2,...,0,0,1,3,1,1,1,1,0,0
HPL,0,0,0,17,24,0,1,3,0,0,...,2,1,0,0,0,0,0,0,2,1
MWS,0,0,1,9,0,0,0,1,2,0,...,0,0,0,0,0,0,0,0,0,0
SUMA,1,2,1,48,24,2,2,6,5,2,...,2,1,1,3,1,1,1,1,2,1


In [22]:
summa=[pivot_col.loc['EAP'].sum(), pivot_col.loc['HPL'].sum(), 
       pivot_col.loc['MWS'].sum(), pivot_col.loc['SUMA'].sum()]
pivot_col['summa']=summa
pivot_col.head()

,aaem,aback,abaft,abandon,abaout,abas,abash,abat,abbey,abbrevi,...,æneid,ærial,æronaut,ærostat,æschylus,élite,émeut,οἶδα,υπνος,summa
EAP,1,2,0,22,0,2,1,2,3,2,...,0,1,3,1,1,1,1,0,0,87765
HPL,0,0,0,17,24,0,1,3,0,0,...,1,0,0,0,0,0,0,2,1,74269
MWS,0,0,1,9,0,0,0,1,2,0,...,0,0,0,0,0,0,0,0,0,73160
SUMA,1,2,1,48,24,2,2,6,5,2,...,1,1,3,1,1,1,1,2,1,235194


In [23]:
# Create probability of author text knowing that a word was used
pivot_part=pivot_col
pivot_part.loc['EAP']=pivot_col.loc['EAP']/pivot_col.loc['SUMA']
pivot_part.loc['HPL']=pivot_col.loc['HPL']/pivot_col.loc['SUMA']
pivot_part.loc['MWS']=pivot_col.loc['MWS']/pivot_col.loc['SUMA']
pivot_part=pivot_part.loc[['EAP', 'HPL', 'MWS']]
# Delete unique words
pivot_part=pivot_part.loc[:, (pivot_part!=1).all(axis=0)]
pivot_part.head()

,abandon,abash,abat,abbey,abdic,aberr,abhor,abhorr,abil,abject,...,younger,youngest,your,youth,zeal,zenith,zest,zigzag,zone,summa
EAP,0.458333,0.5,0.333333,0.6,0.142857,0.166667,0.058824,0.111111,0.789474,0.333333,...,0.272727,0.2,0.534884,0.101562,0.117647,0.4,0.2,0.4,0.666667,0.373160
HPL,0.354167,0.5,0.500000,0.0,0.000000,0.666667,0.235294,0.555556,0.052632,0.000000,...,0.000000,0.4,0.069767,0.429688,0.470588,0.6,0.2,0.6,0.333333,0.315778
MWS,0.187500,0.0,0.166667,0.4,0.857143,0.166667,0.705882,0.333333,0.157895,0.666667,...,0.727273,0.4,0.395349,0.468750,0.411765,0.0,0.6,0.0,0.000000,0.311062


In [24]:
# It will be easier to work this way
eap_dict=pivot_part.loc['EAP'].to_dict()
hpl_dict=pivot_part.loc['HPL'].to_dict()
mws_dict=pivot_part.loc['MWS'].to_dict()
eap_dict['word']

0.44859813084112149

In [25]:
# Create author score 
def ind_val_eap(listn):
    quant=0
    for word in listn:
        try:
            quant+=eap_dict[word]
        except KeyError:
            quant+=0
    return quant

def ind_val_hpl(listn):
    quant=0
    for word in listn:
        try:
            quant+=hpl_dict[word]
        except KeyError:
            quant+=0
    return quant

def ind_val_mws(listn):
    quant=0
    for word in listn:
        try:
            quant+=mws_dict[word]
        except KeyError:
            quant+=0
    return quant

In [26]:
# Add index of author
df_train['mws_index']=df_train['cleaned_text'].apply(ind_val_mws)/df_train['length']
df_train['eap_index']=df_train['cleaned_text'].apply(ind_val_eap)/df_train['length']
df_train['hpl_index']=df_train['cleaned_text'].apply(ind_val_hpl)/df_train['length']
df_train.head(n=3)

,id,text,author,cleaned_text,cleaned_text_string,length,num_words,num_unique_words,num_punctuations,num_words_upper,num_words_title,mean_word_len,num_stopwords,mws_index,eap_index,hpl_index
0,id26305,"This process, however, afforded me no means of...",EAP,"[this, process, howev, afford, mean, ascertain...",this process howev afford mean ascertain dimen...,145,41,35,7,2,3,4.658537,19,0.035935,0.074388,0.034504
1,id17569,It never once occurred to me that the fumbling...,HPL,"[it, never, occur, fumbl, might, mere, mistak]",it never occur fumbl might mere mistak,38,14,14,1,0,1,4.142857,8,0.035860,0.060980,0.034739
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,"[in, left, hand, gold, snuff, box, caper, hill...",in left hand gold snuff box caper hill cut man...,116,36,32,5,0,1,4.583333,16,0.026900,0.047832,0.037337


In [27]:
#Transform authors' names to numeric
df_train['author']=df_train['author'].astype('category')
df_train['author2']=df_train['author'].cat.codes
# Create different features 
df_train.head(n=3)
mid = df_train['author2']
df_train.drop(labels=['author2'], axis=1,inplace = True)
df_train.insert(0, 'author2', mid)
df_train.head()

,author2,id,text,author,cleaned_text,cleaned_text_string,length,num_words,num_unique_words,num_punctuations,num_words_upper,num_words_title,mean_word_len,num_stopwords,mws_index,eap_index,hpl_index
0,0,id26305,"This process, however, afforded me no means of...",EAP,"[this, process, howev, afford, mean, ascertain...",this process howev afford mean ascertain dimen...,145,41,35,7,2,3,4.658537,19,0.035935,0.074388,0.034504
1,1,id17569,It never once occurred to me that the fumbling...,HPL,"[it, never, occur, fumbl, might, mere, mistak]",it never occur fumbl might mere mistak,38,14,14,1,0,1,4.142857,8,0.035860,0.060980,0.034739
2,0,id11008,"In his left hand was a gold snuff box, from wh...",EAP,"[in, left, hand, gold, snuff, box, caper, hill...",in left hand gold snuff box caper hill cut man...,116,36,32,5,0,1,4.583333,16,0.026900,0.047832,0.037337
3,2,id27763,How lovely is spring As we looked from Windsor...,MWS,"[how, love, spring, as, look, windsor, terrac,...",how love spring as look windsor terrac sixteen...,144,34,32,4,0,4,5.088235,13,0.071850,0.033438,0.033601
4,1,id12958,"Finding nothing else, not even gold, the Super...",HPL,"[find, noth, els, even, gold, superintend, aba...",find noth els even gold superintend abandon at...,102,27,25,4,0,2,5.481481,11,0.036859,0.056661,0.043735


In [28]:
author_mapping_dict = {'EAP':0, 'HPL':1, 'MWS':2}
train_y = df_train['author'].map(author_mapping_dict)
train_id = df_train['id'].values
test_id = df_test['id'].values
cols_to_drop = ['id', 'text']
train_X = df_train.drop(cols_to_drop+['author'], axis=1)
test_X = df_test.drop(cols_to_drop, axis=1)
tfidf_vec = TfidfVectorizer(stop_words='english', ngram_range=(1,3))
full_tfidf = tfidf_vec.fit_transform(df_train['text'].values.tolist() + df_test['text'].values.tolist())
train_tfidf = tfidf_vec.transform(df_train['text'].values.tolist())
test_tfidf = tfidf_vec.transform(df_test['text'].values.tolist())

In [29]:
def runMNB(train_X, train_y, test_X, test_y, test_X2):
    model = naive_bayes.MultinomialNB()
    model.fit(train_X, train_y)
    pred_test_y = model.predict_proba(test_X)
    pred_test_y2 = model.predict_proba(test_X2)
    return pred_test_y, pred_test_y2, model

In [30]:
cv_scores = []
pred_full_test = 0
pred_train = np.zeros([df_train.shape[0], 3])
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
for dev_index, val_index in kf.split(train_X):
    dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_tfidf)
    pred_full_test = pred_full_test + pred_test_y
    pred_train[val_index,:] = pred_val_y
    cv_scores.append(metrics.log_loss(val_y, pred_val_y))
print("Mean cv score : ", np.mean(cv_scores))
pred_full_test = pred_full_test / 5.

Mean cv score :  0.842216198361


In [31]:
n_comp = 20
svd_obj = TruncatedSVD(n_components=n_comp, algorithm='arpack')
svd_obj.fit(full_tfidf)
train_svd = pd.DataFrame(svd_obj.transform(train_tfidf))
test_svd = pd.DataFrame(svd_obj.transform(test_tfidf))
    
train_svd.columns = ['svd_word_'+str(i) for i in range(n_comp)]
test_svd.columns = ['svd_word_'+str(i) for i in range(n_comp)]
df_train = pd.concat([df_train, train_svd], axis=1)
df_test = pd.concat([df_test, test_svd], axis=1)
del full_tfidf, train_tfidf, test_tfidf, train_svd, test_svd

In [32]:
### Fit transform the count vectorizer ###
tfidf_vec = CountVectorizer(stop_words='english', ngram_range=(1,3))
tfidf_vec.fit(df_train['text'].values.tolist() + df_test['text'].values.tolist())
train_tfidf = tfidf_vec.transform(df_train['text'].values.tolist())
test_tfidf = tfidf_vec.transform(df_test['text'].values.tolist())

In [33]:
cv_scores = []
pred_full_test = 0
pred_train = np.zeros([df_train.shape[0], 3])
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
for dev_index, val_index in kf.split(train_X):
    dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y, val_X, val_y, test_tfidf)
    pred_full_test = pred_full_test + pred_test_y
    pred_train[val_index,:] = pred_val_y
    cv_scores.append(metrics.log_loss(val_y, pred_val_y))
print("Mean cv score : ", np.mean(cv_scores))
pred_full_test = pred_full_test / 5.

# add the predictions as new features #
df_train["nb_cvec_eap"] = pred_train[:,0]
df_train["nb_cvec_hpl"] = pred_train[:,1]
df_train["nb_cvec_mws"] = pred_train[:,2]
df_test["nb_cvec_eap"] = pred_full_test[:,0]
df_test["nb_cvec_hpl"] = pred_full_test[:,1]
df_test["nb_cvec_mws"] = pred_full_test[:,2]

Mean cv score :  0.450918416166


In [34]:
# Create data set
ds_train=df_train.values
X=ds_train[:, 6:]
Y=ds_train[:, 0]
seed=7
test_size=0.3
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X,Y, test_size=test_size, random_state=seed)

In [35]:
# fit model
xg_train=xgb.DMatrix(X_train, label=y_train)
xg_test=xgb.DMatrix(X_test, label=y_test)
xg_t=xgb.DMatrix(X, label=Y)
param={}
param['objective'] = 'multi:softmax'
param['eta'] = 0.3
param['max_depth'] = 3
param['silent'] = 1
param['num_class'] = 3
param['eval_metric']= "mlogloss"
watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 70
bst = xgb.train(param, xg_train, num_round, watchlist)
# get prediction
pred = bst.predict(xg_test)
error_rate = np.sum(pred != y_test) / y_test.shape[0]
print('Test error using softmax = {}'.format(error_rate))

[0]	train-mlogloss:0.835526	test-mlogloss:0.83777
[1]	train-mlogloss:0.684008	test-mlogloss:0.687812
[2]	train-mlogloss:0.586673	test-mlogloss:0.591944
[3]	train-mlogloss:0.521145	test-mlogloss:0.527776
[4]	train-mlogloss:0.475735	test-mlogloss:0.483269
[5]	train-mlogloss:0.442544	test-mlogloss:0.451403
[6]	train-mlogloss:0.419062	test-mlogloss:0.4292
[7]	train-mlogloss:0.401388	test-mlogloss:0.412859
[8]	train-mlogloss:0.386982	test-mlogloss:0.400323
[9]	train-mlogloss:0.376089	test-mlogloss:0.390533
[10]	train-mlogloss:0.366897	test-mlogloss:0.38265
[11]	train-mlogloss:0.359258	test-mlogloss:0.376604
[12]	train-mlogloss:0.353014	test-mlogloss:0.37199
[13]	train-mlogloss:0.34766	test-mlogloss:0.368182
[14]	train-mlogloss:0.342403	test-mlogloss:0.36475
[15]	train-mlogloss:0.338286	test-mlogloss:0.361916
[16]	train-mlogloss:0.335046	test-mlogloss:0.360015
[17]	train-mlogloss:0.331689	test-mlogloss:0.357469
[18]	train-mlogloss:0.327961	test-mlogloss:0.355265
[19]	train-mlogloss:0.324831	

In [36]:
# do the same thing again, but output probabilities
param['objective'] = 'multi:softprob'
bstp = xgb.train(param, xg_train, num_round, watchlist)
# Note: this convention has been changed since xgboost-unity
# get prediction, this is in 1D array, need reshape to (ndata, nclass)
pred_prob = bstp.predict(xg_test).reshape(y_test.shape[0], 3)
pred_label = np.argmax(pred_prob, axis=1)
error_rate = np.sum(pred_label != y_test) / y_test.shape[0]
print('Test error using softprob = {}'.format(error_rate))

[0]	train-mlogloss:0.835526	test-mlogloss:0.83777
[1]	train-mlogloss:0.684008	test-mlogloss:0.687812
[2]	train-mlogloss:0.586673	test-mlogloss:0.591944
[3]	train-mlogloss:0.521145	test-mlogloss:0.527776
[4]	train-mlogloss:0.475735	test-mlogloss:0.483269
[5]	train-mlogloss:0.442544	test-mlogloss:0.451403
[6]	train-mlogloss:0.419062	test-mlogloss:0.4292
[7]	train-mlogloss:0.401388	test-mlogloss:0.412859
[8]	train-mlogloss:0.386982	test-mlogloss:0.400323
[9]	train-mlogloss:0.376089	test-mlogloss:0.390533
[10]	train-mlogloss:0.366897	test-mlogloss:0.38265
[11]	train-mlogloss:0.359258	test-mlogloss:0.376604
[12]	train-mlogloss:0.353014	test-mlogloss:0.37199
[13]	train-mlogloss:0.34766	test-mlogloss:0.368182
[14]	train-mlogloss:0.342403	test-mlogloss:0.36475
[15]	train-mlogloss:0.338286	test-mlogloss:0.361916
[16]	train-mlogloss:0.335046	test-mlogloss:0.360015
[17]	train-mlogloss:0.331689	test-mlogloss:0.357469
[18]	train-mlogloss:0.327961	test-mlogloss:0.355265
[19]	train-mlogloss:0.324831	

In [37]:
# do the same thing again, but output probabilities
param['objective'] = 'multi:softprob'
bstp = xgb.train(param, xg_t, num_round, watchlist)
# Note: this convention has been changed since xgboost-unity
# get prediction, this is in 1D array, need reshape to (ndata, nclass)
pred_prob = bstp.predict(xg_t).reshape(Y.shape[0], 3)
pred_label = np.argmax(pred_prob, axis=1)
error_rate = np.sum(pred_label != Y) / Y.shape[0]
print('Test error using softprob = {}'.format(error_rate))

[0]	train-mlogloss:0.83624	test-mlogloss:0.835877
[1]	train-mlogloss:0.685467	test-mlogloss:0.684348
[2]	train-mlogloss:0.588694	test-mlogloss:0.587736
[3]	train-mlogloss:0.523663	test-mlogloss:0.522296
[4]	train-mlogloss:0.478455	test-mlogloss:0.476666
[5]	train-mlogloss:0.446339	test-mlogloss:0.444137
[6]	train-mlogloss:0.423001	test-mlogloss:0.420648
[7]	train-mlogloss:0.404639	test-mlogloss:0.402578
[8]	train-mlogloss:0.391304	test-mlogloss:0.389033
[9]	train-mlogloss:0.380503	test-mlogloss:0.378182
[10]	train-mlogloss:0.371494	test-mlogloss:0.369497
[11]	train-mlogloss:0.36346	test-mlogloss:0.362008
[12]	train-mlogloss:0.358119	test-mlogloss:0.356453
[13]	train-mlogloss:0.352523	test-mlogloss:0.35156
[14]	train-mlogloss:0.348598	test-mlogloss:0.347614
[15]	train-mlogloss:0.343744	test-mlogloss:0.343318
[16]	train-mlogloss:0.340244	test-mlogloss:0.34006
[17]	train-mlogloss:0.337073	test-mlogloss:0.336992
[18]	train-mlogloss:0.334195	test-mlogloss:0.334553
[19]	train-mlogloss:0.3318

In [38]:
df_test.head(n=3)

,id,text,svd_word_0,svd_word_1,svd_word_2,svd_word_3,svd_word_4,svd_word_5,svd_word_6,svd_word_7,...,svd_word_13,svd_word_14,svd_word_15,svd_word_16,svd_word_17,svd_word_18,svd_word_19,nb_cvec_eap,nb_cvec_hpl,nb_cvec_mws
0,id02310,"Still, as I urged our leaving Ireland with suc...",0.024516,-0.010185,0.001168,-0.005363,-0.013319,-0.003444,-0.002816,0.003240,...,-0.011837,0.036064,-0.016591,-0.025580,-0.018785,0.031289,-0.047220,0.021018,0.000595,0.978387
1,id24541,"If a fire wanted fanning, it could readily be ...",0.022294,-0.011968,-0.001596,-0.004478,-0.012514,-0.000641,-0.009725,-0.000215,...,-0.004397,-0.000020,-0.008583,0.006335,-0.004216,0.001810,0.001767,0.999985,0.000009,0.000006
2,id00134,And when they had broken down the frail door t...,0.016906,-0.008934,0.000240,-0.006892,-0.008843,0.004787,-0.005058,-0.004598,...,0.006063,-0.003324,-0.009452,0.013239,0.004852,-0.007478,0.002786,0.217325,0.782527,0.000148


In [39]:
df_test['cleaned_text'] = df_test.text.apply(tokenize_stem)
df_test['cleaned_text_string'] = df_test.cleaned_text.apply(' '.join)
df_test.head(n=3)

,id,text,svd_word_0,svd_word_1,svd_word_2,svd_word_3,svd_word_4,svd_word_5,svd_word_6,svd_word_7,...,svd_word_15,svd_word_16,svd_word_17,svd_word_18,svd_word_19,nb_cvec_eap,nb_cvec_hpl,nb_cvec_mws,cleaned_text,cleaned_text_string
0,id02310,"Still, as I urged our leaving Ireland with suc...",0.024516,-0.010185,0.001168,-0.005363,-0.013319,-0.003444,-0.002816,0.003240,...,-0.016591,-0.025580,-0.018785,0.031289,-0.047220,0.021018,0.000595,0.978387,"[still, i, urg, leav, ireland, inquietud, impa...",still i urg leav ireland inquietud impati fath...
1,id24541,"If a fire wanted fanning, it could readily be ...",0.022294,-0.011968,-0.001596,-0.004478,-0.012514,-0.000641,-0.009725,-0.000215,...,-0.008583,0.006335,-0.004216,0.001810,0.001767,0.999985,0.000009,0.000006,"[if, fire, want, fan, could, readili, fan, new...",if fire want fan could readili fan newspap gov...
2,id00134,And when they had broken down the frail door t...,0.016906,-0.008934,0.000240,-0.006892,-0.008843,0.004787,-0.005058,-0.004598,...,-0.009452,0.013239,0.004852,-0.007478,0.002786,0.217325,0.782527,0.000148,"[and, broken, frail, door, found, two, clean, ...",and broken frail door found two clean pick hum...


In [40]:
df_test['length']=df_test['cleaned_text_string'].apply(len)
df_test["num_words"] = df_test["text"].apply(lambda x: len(str(x).split()))
df_test["num_unique_words"] = df_test["text"].apply(lambda x: len(set(str(x).split())))
df_test["num_punctuations"] =df_test['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
df_test["num_words_upper"] = df_test["text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
df_test["num_words_title"] = df_test["text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
df_test["mean_word_len"] = df_test["text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
df_test["num_stopwords"] = df_test["text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
df_test.head(n=3)

,id,text,svd_word_0,svd_word_1,svd_word_2,svd_word_3,svd_word_4,svd_word_5,svd_word_6,svd_word_7,...,cleaned_text,cleaned_text_string,length,num_words,num_unique_words,num_punctuations,num_words_upper,num_words_title,mean_word_len,num_stopwords
0,id02310,"Still, as I urged our leaving Ireland with suc...",0.024516,-0.010185,0.001168,-0.005363,-0.013319,-0.003444,-0.002816,0.003240,...,"[still, i, urg, leav, ireland, inquietud, impa...",still i urg leav ireland inquietud impati fath...,67,19,19,3,1,3,4.842105,9
1,id24541,"If a fire wanted fanning, it could readily be ...",0.022294,-0.011968,-0.001596,-0.004478,-0.012514,-0.000641,-0.009725,-0.000215,...,"[if, fire, want, fan, could, readili, fan, new...",if fire want fan could readili fan newspap gov...,181,62,49,7,1,3,4.338710,33
2,id00134,And when they had broken down the frail door t...,0.016906,-0.008934,0.000240,-0.006892,-0.008843,0.004787,-0.005058,-0.004598,...,"[and, broken, frail, door, found, two, clean, ...",and broken frail door found two clean pick hum...,114,33,30,3,0,1,4.757576,15


In [41]:
df_test['mws_index']=df_test['cleaned_text'].apply(ind_val_mws)/df_test['length']
df_test['eap_index']=df_test['cleaned_text'].apply(ind_val_eap)/df_test['length']
df_test['hpl_index']=df_test['cleaned_text'].apply(ind_val_hpl)/df_test['length']
df_test.head(n=3)
df_test.head(n=3)

,id,text,svd_word_0,svd_word_1,svd_word_2,svd_word_3,svd_word_4,svd_word_5,svd_word_6,svd_word_7,...,num_words,num_unique_words,num_punctuations,num_words_upper,num_words_title,mean_word_len,num_stopwords,mws_index,eap_index,hpl_index
0,id02310,"Still, as I urged our leaving Ireland with suc...",0.024516,-0.010185,0.001168,-0.005363,-0.013319,-0.003444,-0.002816,0.003240,...,19,19,3,1,3,4.842105,9,0.071473,0.036315,0.026541
1,id24541,"If a fire wanted fanning, it could readily be ...",0.022294,-0.011968,-0.001596,-0.004478,-0.012514,-0.000641,-0.009725,-0.000215,...,62,49,7,1,3,4.338710,33,0.035932,0.062884,0.039306
2,id00134,And when they had broken down the frail door t...,0.016906,-0.008934,0.000240,-0.006892,-0.008843,0.004787,-0.005058,-0.004598,...,33,30,3,0,1,4.757576,15,0.027486,0.055142,0.057723


In [42]:
del df_test['cleaned_text']
del df_test['cleaned_text_string']

In [43]:
cols=(df_train.columns.tolist())[6:]
[item for item in df_train.columns.tolist() if item not in df_test.columns.tolist()]

['author2', 'author', 'cleaned_text', 'cleaned_text_string']

In [44]:
df_test.head()

,id,text,svd_word_0,svd_word_1,svd_word_2,svd_word_3,svd_word_4,svd_word_5,svd_word_6,svd_word_7,...,num_words,num_unique_words,num_punctuations,num_words_upper,num_words_title,mean_word_len,num_stopwords,mws_index,eap_index,hpl_index
0,id02310,"Still, as I urged our leaving Ireland with suc...",0.024516,-0.010185,0.001168,-0.005363,-0.013319,-0.003444,-0.002816,0.003240,...,19,19,3,1,3,4.842105,9,0.071473,0.036315,0.026541
1,id24541,"If a fire wanted fanning, it could readily be ...",0.022294,-0.011968,-0.001596,-0.004478,-0.012514,-0.000641,-0.009725,-0.000215,...,62,49,7,1,3,4.338710,33,0.035932,0.062884,0.039306
2,id00134,And when they had broken down the frail door t...,0.016906,-0.008934,0.000240,-0.006892,-0.008843,0.004787,-0.005058,-0.004598,...,33,30,3,0,1,4.757576,15,0.027486,0.055142,0.057723
3,id27757,While I was thinking how I should possibly man...,0.013408,-0.007515,-0.000154,-0.004020,-0.004521,0.002712,-0.004220,-0.002882,...,41,34,5,2,3,4.463415,19,0.024369,0.065057,0.055736
4,id04081,I am not sure to what limit his knowledge may ...,0.012565,-0.003185,-0.000719,-0.001152,0.000930,-0.006110,0.001690,-0.002149,...,11,11,1,1,1,3.909091,6,0.025615,0.070948,0.028437


In [49]:
ds_test=df_test[cols].values

,length,num_words,num_unique_words,num_punctuations,num_words_upper,num_words_title,mean_word_len,num_stopwords,mws_index,eap_index,...,svd_word_13,svd_word_14,svd_word_15,svd_word_16,svd_word_17,svd_word_18,svd_word_19,nb_cvec_eap,nb_cvec_hpl,nb_cvec_mws
0,67,19,19,3,1,3,4.842105,9,0.071473,0.036315,...,-0.011837,0.036064,-0.016591,-0.025580,-0.018785,0.031289,-0.047220,2.101817e-02,5.947015e-04,9.783871e-01
1,181,62,49,7,1,3,4.338710,33,0.035932,0.062884,...,-0.004397,-0.000020,-0.008583,0.006335,-0.004216,0.001810,0.001767,9.999853e-01,8.925621e-06,5.822058e-06
2,114,33,30,3,0,1,4.757576,15,0.027486,0.055142,...,0.006063,-0.003324,-0.009452,0.013239,0.004852,-0.007478,0.002786,2.173252e-01,7.825270e-01,1.477923e-04
3,124,41,34,5,2,3,4.463415,19,0.024369,0.065057,...,0.004783,-0.006865,-0.007960,0.006763,0.002540,-0.004558,-0.000728,7.535912e-01,2.464078e-01,1.067551e-06
4,32,11,11,1,1,1,3.909091,6,0.025615,0.070948,...,-0.001825,0.000123,-0.006504,0.002533,-0.004004,-0.002902,0.000315,9.709496e-01,2.182445e-02,7.225903e-03
5,122,32,28,8,0,3,5.343750,14,0.031787,0.058253,...,-0.001080,-0.005546,-0.005505,-0.001588,-0.000291,0.000240,-0.000189,9.305380e-01,6.897514e-02,4.869025e-04
6,32,13,11,2,0,1,4.307692,9,0.026596,0.062687,...,0.010832,-0.005349,-0.013915,-0.038165,0.011051,0.058084,-0.015136,6.581746e-01,3.115806e-01,3.024479e-02
7,84,30,23,3,5,5,4.200000,17,0.052639,0.036987,...,0.004515,0.065197,0.002960,-0.028084,-0.009015,-0.010479,0.027816,5.813017e-04,5.729142e-02,9.421273e-01
8,90,28,22,5,0,1,5.178571,14,0.019099,0.064194,...,0.010716,0.005200,0.014222,-0.024173,-0.034284,-0.045892,-0.038832,9.991619e-01,8.371526e-04,9.051063e-07
9,34,8,8,2,0,1,4.375000,2,0.038848,0.057184,...,0.003483,-0.007926,0.017838,-0.003067,-0.006143,0.001231,0.018030,8.102007e-01,7.223093e-02,1.175684e-01


In [53]:
x_t=ds_test[:, :]
y_t=df_test['id'].values
xg_t=xgb.DMatrix(x_t)
pred_prob = bstp.predict(xg_t).reshape(y_t.shape[0], 3)
pred_label = np.argmax(pred_prob, axis=1)

In [55]:
print(pred_prob)

[[  1.89457480e-02   2.00906652e-03   9.79045212e-01]
 [  9.97643292e-01   1.61947298e-03   7.37223891e-04]
 [  1.10446438e-01   8.87890637e-01   1.66295422e-03]
 ..., 
 [  9.61848855e-01   1.63717140e-02   2.17794720e-02]
 [  1.42889163e-02   1.98633247e-03   9.83724833e-01]
 [  6.57032570e-03   9.93166029e-01   2.63663038e-04]]


In [56]:
export=pd.DataFrame(pred_prob)
export.insert(loc=0, column='id', value=y_t)
export.columns=['id','EAP', 'HPL', 'MWS']
export.head()

,id,EAP,HPL,MWS
0,id02310,0.018946,0.002009,0.979045
1,id24541,0.997643,0.001619,0.000737
2,id00134,0.110446,0.887891,0.001663
3,id27757,0.882366,0.116390,0.001244
4,id04081,0.980214,0.012503,0.007283


In [57]:
len(export)

8392

In [58]:
export[export['id']=='id23301']

,id,EAP,HPL,MWS
6106,id23301,0.136131,0.267906,0.595963


In [59]:
export.to_csv(path_or_buf="../data/export.csv", index=False)